In [4]:
!pip install konlpy

from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, Embedding, Bidirectional, Concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

In [6]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# Data Frame Create
story_df = pd.read_csv('/content/drive/MyDrive/my_ws/project/Final/Data/story.csv', encoding='cp949')

# 1. 정규식을 사용해서 특수문자, 숫자 제거
# story_df의 각 열에 대해서 정규 표현식을 사용하여 문자열 치환을 수행하는 반복문입니다.
for i in range(10):  # 0부터 9까지의 열 인덱스
    story_df[str(i)] = story_df[str(i)].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]', '')

story_df.drop(['결론'], axis=1, inplace=True)

# 결과를 저장할 빈 데이터프레임 리스트를 초기화합니다.
dataframes = []

# story_df에서 9열까지 있다고 가정합니다 (0부터 9까지, 총 10개의 열).
# 0열부터 8열까지를 input으로 하고, 그 다음 열을 output으로 하는 데이터프레임을 생성합니다.
for i in range(9):  # 0부터 8까지 (9는 포함되지 않음)
    # 각 행에 대해 원하는 열을 공백으로 구분하여 합치기 위한 람다 함수를 apply 메서드에 사용합니다.
    input_column = story_df.iloc[:, :(i + 1)].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
    output_column = story_df.iloc[:, i + 1]

    # 새로운 데이터프레임을 만들고 dataframes 리스트에 추가합니다.
    df = pd.DataFrame({'input': input_column, 'output': output_column})
    dataframes.append(df)

# 모든 데이터프레임을 세로 방향으로 합쳐서 result_df를 생성합니다.
result_df = pd.concat(dataframes, ignore_index=True)

# 결과를 출력합니다.
result_df

x_data_df = result_df['input'];
t_data_df = result_df['output'];

x_data_train, x_data_test, t_data_train, t_data_test = \
train_test_split(x_data_df,
                 t_data_df,
                 test_size=0.2,
                 random_state=42)

question, answer = list(x_data_train), list(t_data_train)
# 챗봇 데이터 출력
for i in range(5):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 알렉스는 특별한 능력을 가진 청년으로 그의 능력은 남들이 볼 수 없는 꿈의 세계를 볼 수 있다는 것입니다
A : 그는 이 능력을 숨기며 평범한 삶을 살고 있었지만 어느 날 그의 꿈에 미지의 여성 에밀리가 나타납니다

Q : 영화는 평범한 삶을 살던 주인공 제이크의 이야기로 시작된다 제이크는 어느 날 잠에서 깨어나면 다른 세상에 떨어진 것을 발견한다 그곳은 고대 시대로 제이크는 거대한 동물들 사이에서 생존을 위한 투쟁을 시작한다 그러나 그는 점차 그 세상의 독특한 문화와 사람들에게 매료되어 간다
A : 그리고 그는 고대 세상의 여왕 레나와 사랑에 빠진다

Q : 이야기는 외계 행성에 살고 있는 과학자 훈이의 일상에서 시작된다 훈은 외계 생명체 연구를 통해 새로운 에너지원을 발견하게 된다 그러나 그의 발견은 비밀스럽게 행성을 지배하는 조직의 이목을 끈다
A : 조직은 훈의 연구를 이용해 행성을 지배하려는 계획을 세운다

Q : 조용한 작은 마을에서 살던 청년 톰은 어느 날 자신이 마법의 힘을 가진 사람임을 알게 된다 이는 그의 할아버지로부터 전해진 비밀로 그는 금방 이 힘을 제어하는 법을 배우지 못한다 그의 마법은 자주 부주의하게 행동하며 톰을 곤경에 빠뜨린다 그러나 그는 이 힘을 바른 방향으로 사용하기 위해 노력하며 자신만의 능력을 발견한다 그는 주변 사람들의 도움을 받아 마법의 힘을 제어하는 법을 배우고 그 힘을 사용해 사람들을 도와주기 시작한다 그의 마법은 마을 사람들에게 기쁨과 행복을 주며 그는 마을의 영웅이 된다 그러나 어둠의 마법사가 그의 마법의 힘을 원하며 마을을 위협하기 시작한다
A : 톰은 마을을 지키기 위해 어둠의 마법사와 대결하며 그의 진정한 용기를 보여준다

Q : 서울의 한 가운데 잘 알려지지 않은 작은 카페에서 주인공 김수현은 평범한 일상을 살아가고 있었습니다 그러던 어느 날 그의 카페에 미스터리한 여성 유나가 찾아오면서 이야기는 새로운 방향으로 흘러가게 됩니다 유나는 과거의 아픈 기억을 갖고 있었고 그로 인해 현재에 갇혀 살고 있

<ipython-input-6-c4564fd99f46>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  story_df[str(i)] = story_df[str(i)].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]', '')


In [7]:
# 데이터 개수
len(question)

720

In [8]:
# 형태소분석 함수
def pos_tag(sentences):

    # KoNLPy 형태소분석기 설정
    tagger = Okt()

    # 문장 품사 변수 초기화
    sentences_pos = []

    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)

        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)

    return sentences_pos

In [9]:
# 형태소분석 수행
question = pos_tag(question)
answer = pos_tag(answer)

# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()

Q : 알렉스 는 특별한 능력 을 가진 청년 으로 그 의 능력 은 남 들 이 볼 수 없는 꿈 의 세계 를 볼 수 있다는 것 입니다
A : 그 는 이 능력 을 숨기 며 평범한 삶 을 살 고 있었지만 어느 날 그 의 꿈 에 미지 의 여성 에밀리 가 나타납니다

Q : 영화 는 평범한 삶 을 살던 주인공 제이크 의 이야기 로 시작 된다 제이크 는 어느 날 잠 에서 깨어나면 다른 세상 에 떨어진 것 을 발견 한다 그 곳 은 고대 시대 로 제이크 는 거대한 동물 들 사이 에서 생존 을 위 한 투쟁 을 시작 한 다 그러나 그 는 점차 그 세상 의 독특한 문화 와 사람 들 에게 매료 되어 간다
A : 그리고 그 는 고대 세상 의 여왕 레나 와 사랑 에 빠진다

Q : 이야기 는 외계 행성 에 살 고 있는 과학자 훈이 의 일상 에서 시작 된다 훈 은 외계 생명체 연구 를 통해 새로운 에너지 원 을 발견 하게 된다 그러나 그 의 발견 은 비밀 스럽게 행성 을 지배 하는 조직 의 이목 을 끈다
A : 조직 은 훈 의 연구 를 이용 해 행성 을 지배 하려는 계획 을 세운다

Q : 조용한 작은 마을 에서 살던 청년 톰 은 어느 날 자신 이 마법 의 힘 을 가진 사람 임 을 알 게 된다 이는 그 의 할아버지 로부터 전해진 비밀 로 그 는 금방 이 힘 을 제어 하는 법 을 배우지 못 한다 그 의 마법 은 자주 부주의하게 행동 하며 톰 을 곤경 에 빠뜨린다 그러나 그 는 이 힘 을 바른 방향 으로 사용 하기 위해 노력 하며 자신 만의 능력 을 발견 한다 그 는 주변 사람 들 의 도움 을 받아 마법 의 힘 을 제어 하는 법 을 배우고 그 힘 을 사용 해 사람 들 을 도와주기 시작 한 다 그 의 마법 은 마을 사람 들 에게 기쁨 과 행복 을 주며 그 는 마을 의 영웅 이 된다 그러나 어둠 의 마법사 가 그 의 마법 의 힘 을 원하며 마을 을 위협 하기 시작 한 다
A : 톰 은 마을 을 지키기 위해 어둠 의 마법사 와 대결 하며 그 의 진정한 용기 를 보여준다

Q : 서울 의 한 가운데

In [10]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)
sentences.extend(answer)

words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [11]:
# 단어 개수
len(words)

2592

In [12]:
# 단어 출력
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '처절한',
 '중지',
 '박현식',
 '주인공',
 '대대로',
 '관심',
 '반대',
 '로간',
 '도와주겠다고',
 '놀랍니다',
 '악의',
 '숨기기',
 '침략',
 '거슬러가',
 '행',
 '전사',
 '바뀌어',
 '넘나',
 '소원',
 '빠트리',
 '취하',
 '나타나자',
 '열쇠',
 '잡는',
 '사고',
 '성장합니다',
 '만남',
 '시간여행',
 '잡기',
 '번',
 '매튜',
 '진실',
 '전쟁',
 '가까워진다',
 '결국',
 '달라',
 '청',
 '포기',
 '안게',
 '마법사',
 '깨어나면',
 '서연',
 '리나',
 '해준다',
 '상황',
 '성공하며',
 '싸울',
 '지키기로',
 '테드',
 '연결고리',
 '부딪힌',
 '음모',
 '직접',
 '대는',
 '외계인',
 '몰두하던',
 '얻었지만',
 '찾으려고',
 '있었고',
 '프로그램',
 '참여',
 '하여금',
 '가는',
 '돌리기',
 '여왕',
 '물고기',
 '크게',
 '만나고',
 '만든',
 '거절',
 '받아들이고',
 '조립',
 '글쓰기',
 '마주',
 '빗나가게',
 '두려워하지만',
 '조차',
 '모은',
 '위협',
 '오지',
 '작',
 '다름없',
 '집니다',
 '묘',
 '찾아냅니다',
 '만듭니',
 '제공',
 '노리는',
 '해보게',
 '마지막',
 '이루어주지',
 '서는',
 '영수',
 '왕관',
 '모험가',
 '어릴',
 '찾아가기',
 '깊어지면서',
 '비밀',
 '감수',
 '되며',
 '스스로',
 '존재',
 '발전',
 '이윤',
 '이라',
 '찾아간다',
 '전통',
 '친한',
 '악행',
 '데이비드',
 '선장',
 '나면서',
 '소중한',
 '수수께끼',
 '방지',
 '또',
 '하기로',
 '인류',
 '버전',
 '거쳐가며',
 '용의',
 '꿈꾸던',
 '배우지',


In [13]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [14]:
# 단어 -> 인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<START>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '처절한': 4,
 '중지': 5,
 '박현식': 6,
 '주인공': 7,
 '대대로': 8,
 '관심': 9,
 '반대': 10,
 '로간': 11,
 '도와주겠다고': 12,
 '놀랍니다': 13,
 '악의': 14,
 '숨기기': 15,
 '침략': 16,
 '거슬러가': 17,
 '행': 18,
 '전사': 19,
 '바뀌어': 20,
 '넘나': 21,
 '소원': 22,
 '빠트리': 23,
 '취하': 24,
 '나타나자': 25,
 '열쇠': 26,
 '잡는': 27,
 '사고': 28,
 '성장합니다': 29,
 '만남': 30,
 '시간여행': 31,
 '잡기': 32,
 '번': 33,
 '매튜': 34,
 '진실': 35,
 '전쟁': 36,
 '가까워진다': 37,
 '결국': 38,
 '달라': 39,
 '청': 40,
 '포기': 41,
 '안게': 42,
 '마법사': 43,
 '깨어나면': 44,
 '서연': 45,
 '리나': 46,
 '해준다': 47,
 '상황': 48,
 '성공하며': 49,
 '싸울': 50,
 '지키기로': 51,
 '테드': 52,
 '연결고리': 53,
 '부딪힌': 54,
 '음모': 55,
 '직접': 56,
 '대는': 57,
 '외계인': 58,
 '몰두하던': 59,
 '얻었지만': 60,
 '찾으려고': 61,
 '있었고': 62,
 '프로그램': 63,
 '참여': 64,
 '하여금': 65,
 '가는': 66,
 '돌리기': 67,
 '여왕': 68,
 '물고기': 69,
 '크게': 70,
 '만나고': 71,
 '만든': 72,
 '거절': 73,
 '받아들이고': 74,
 '조립': 75,
 '글쓰기': 76,
 '마주': 77,
 '빗나가게': 78,
 '두려워하지만': 79,
 '조차': 80,
 '모은': 81,
 '위협': 82,
 '오지': 83,
 '작': 84,
 

In [15]:
# 인덱스 -> 단어
# 모델의 예측 결과인 인덱스를 문장으로 변환시 사용
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '처절한',
 5: '중지',
 6: '박현식',
 7: '주인공',
 8: '대대로',
 9: '관심',
 10: '반대',
 11: '로간',
 12: '도와주겠다고',
 13: '놀랍니다',
 14: '악의',
 15: '숨기기',
 16: '침략',
 17: '거슬러가',
 18: '행',
 19: '전사',
 20: '바뀌어',
 21: '넘나',
 22: '소원',
 23: '빠트리',
 24: '취하',
 25: '나타나자',
 26: '열쇠',
 27: '잡는',
 28: '사고',
 29: '성장합니다',
 30: '만남',
 31: '시간여행',
 32: '잡기',
 33: '번',
 34: '매튜',
 35: '진실',
 36: '전쟁',
 37: '가까워진다',
 38: '결국',
 39: '달라',
 40: '청',
 41: '포기',
 42: '안게',
 43: '마법사',
 44: '깨어나면',
 45: '서연',
 46: '리나',
 47: '해준다',
 48: '상황',
 49: '성공하며',
 50: '싸울',
 51: '지키기로',
 52: '테드',
 53: '연결고리',
 54: '부딪힌',
 55: '음모',
 56: '직접',
 57: '대는',
 58: '외계인',
 59: '몰두하던',
 60: '얻었지만',
 61: '찾으려고',
 62: '있었고',
 63: '프로그램',
 64: '참여',
 65: '하여금',
 66: '가는',
 67: '돌리기',
 68: '여왕',
 69: '물고기',
 70: '크게',
 71: '만나고',
 72: '만든',
 73: '거절',
 74: '받아들이고',
 75: '조립',
 76: '글쓰기',
 77: '마주',
 78: '빗나가게',
 79: '두려워하지만',
 80: '조차',
 81: '모은',
 82: '위협',
 83: '오지',
 84: '작',
 

In [16]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type):

    sentences_index = []

    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []

        # 디코더 입력일 경우 맨 앞에 START 태그 추가
        if type == DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])

        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                # 사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                # 사전에 없는 단어면 OOV 인덱스를 추가
                sentence_index.extend([vocabulary[OOV]])

        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END 태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index += [vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]

        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]

        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)

In [17]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫 번째 인코더 입력 출력 (12시 땡)
x_encoder[0]

array([2096, 1976, 1201, 2357, 2444, 1518, 1604, 2178,  396, 1307, 2357,
        740, 1552, 2190, 2130,  801, 2312,  929, 2562, 1307, 2208,  406,
        801, 2312, 1078, 1236, 1752,    0,    0,    0])

In [18]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 하루 가 또 가네요)
x_decoder[0]


array([   1,  396, 1976, 2130, 2357, 2444,  704,  343, 1202, 1932, 2444,
       1933, 1142, 2516, 1049, 1304,  396, 1307, 2562, 2500, 1093, 1307,
       2268,  847, 2385, 1169,    0,    0,    0,    0])

In [19]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (하루 가 또 가네요 END)
y_decoder[0]

array([ 396, 1976, 2130, 2357, 2444,  704,  343, 1202, 1932, 2444, 1933,
       1142, 2516, 1049, 1304,  396, 1307, 2562, 2500, 1093, 1307, 2268,
        847, 2385, 1169,    2,    0,    0,    0,    0])

In [20]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))

# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [21]:
#--------------------------------------------
# 훈련 모델 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, forward_h, forward_c = LSTM(lstm_hidden_dim, dropout=0.1, recurrent_dropout=0.5, return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [forward_h, forward_c]



#--------------------------------------------
# 훈련 모델 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 100)            259200    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 100)            259200    ['input_2[0][0]']             
                                                                                              

In [22]:
#--------------------------------------------
#  예측 모델 인코더 정의
#--------------------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = Model(encoder_inputs, encoder_states)



#--------------------------------------------
# 예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_forward_h = Input(shape=(lstm_hidden_dim,))
decoder_state_input_forward_c = Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_forward_h,
                         decoder_state_input_forward_c]

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_forward_h, state_forward_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_forward_h, state_forward_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [23]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary):

    sentence = ''

    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence += vocabulary[OOV_INDEX]

        # 빈칸 추가
        sentence += ' '

    return sentence

In [24]:
# Callback 설정
cp_cb = ModelCheckpoint(filepath='./best_model.ckpt',
                        save_weights_only=True,
                        save_best_only=True,
                        monitor='val_acc',
                        verbose=1)

es_cb = EarlyStopping(monitor='val_acc',
                      patience=0,
                      verbose=1,
                      baseline=0.02,
                      mode='min')

# 에폭 반복
for epoch in range(1):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=1,
                        batch_size=64,
                        callbacks=[cp_cb, es_cb],
                        verbose=1
                        )

    # 정확도와 손실 출력
    print('accuracy :', history.history['acc'][-1])
    print('loss :', history.history['loss'][-1])

    # 문장 예측 테스트
    # (3 박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])

    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1)

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()


Total Epoch : 1
12/12 [==============================] - ETA: 0s - loss: 7.7438 - acc: 0.2715

12/12 [==============================] - 10s 238ms/step - loss: 7.7438 - acc: 0.2715
accuracy : 0.2714814841747284
loss : 7.743809700012207
1/1 [==============================] - 0s 381ms/step
<PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> 



In [ ]:
# 모델 저장
encoder_model.save('./model/seq2seq_encoder_model.h5')
decoder_model.save('./model/seq2seq_decoder_model.h5')

# 인덱스 저장
with open('./model/seq2seq_word_to_index.pkl', 'wb') as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/seq2seq_index_to_word.pkl', 'wb') as f:
    pickle.dump(index_to_word, f, pickle.HIGHEST_PROTOCOL)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# 모델 파일 로드
encoder_model = load_model('./model/seq2seq_encoder_model.h5')
decoder_model = load_model('./model/seq2seq_encoder_model.h5')

# 인덱스 파일 로드
with open('./model/seq2seq_word_to_index.pkl', 'rb') as f:
    word_to_index = pickle.load(f)
with open('./model/seq2seq_index_to_word.pkl', 'rb') as f:
    index_to_word = pickle.load(f)

In [25]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)

    return input_seq

In [29]:
# 텍스트 생성
def generate_text(input_seq):

    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))

    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX

    # 인덱스 초기화
    indexs = []

    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_forward_h, state_forward_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)

        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index

        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_forward_h, state_forward_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    return sentence

In [27]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('주인공 재훈은 평범한 생활을 이어가며 살아가고 있었다')
input_seq

array([[   7,  553,  740, 1202, 2330, 2444, 2130,  266,  343, 2230,  421,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]])

In [30]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

1/1 [==============================] - 0s 18ms/step


'<PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> <PADDING> '